<a href="https://colab.research.google.com/github/marcory-hub/hailo/blob/main/DFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Prepair you dataset


1. Dataset Structure:
  
  Make a annotated dataset with this folder and naming structure. If you do not have your own dataset, you can download the [hornet3000+ dataset](https://www.kaggle.com/datasets/marcoryvandijk/vespa-velutina-v-crabro-vespulina-vulgaris). If you want to annotate your own dataset take a look at CVAT or Roboflow to annotate images and save it in YOLO format.

```
data/
  train/
    images/
      image_1.jpg  # Image file
      image_2.png  # Image file
      etc...
    labels/
      image_1.txt  # Label file
      image_2.txt  # Label file
      etc...
  val/
    images/
      image_3.jpg  # Image for validation
      image_4.png  # Image for validation
      ...
    labels/
      image_3.txt  # Label file for validation
      image_4.txt  # Label file for validation
      ...
```

2. data.yaml Configuration (Optional)

  Add `data.yaml` (configuration file used by the training script to locate the data) to folder datasets. This contains:
  - absolute path to train images (train)
  - absolute path validation (val)
  - the number of classes (nc)
  - and the class names (names)

  For example a dataset with 3 types of insects would look like this:
```
train: /content/data/train # path to train images
val: /content/data/val # path to val images
nc: 3
names: ['Vespa_velutina', 'Vespa_crabro', 'Vespula_vulgaris']
```
Make sure to adjust the `nc` (number of classes) and `names` accordingly.

3. Zip Your Dataset (Optional):

  Only if you use own dataset: zip the data folder to a file names `dataset.zip` (or a custom name). On mac use `ditto -c -k --norsrc --keepParent images dataset.zip` to exclude finderfiles from the zipped file.

4. Copy the zipped dataset:
  
  Copy the zipped file with dataset and yaml to your google drive.



## 2. Unzip dataset.zip and rename the folder on google drive

Adjust the names of the `dataset_path` and `dataset_filename` in the boxes on the right.

In [ ]:
## 2. Unzip dataset.zip and rename the folder on google drive
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
import os

# Define Paths with Parameters
dataset_path = "/content/gdrive/MyDrive/vespA/100_1_dataset.zip"  # @param {type:"string"}
dataset_filename = "100_1_sampleSize"  # @param {type:"string"}

# Unzip the Dataset (using the defined path)
!unzip {dataset_path} -d '/content/'

# Rename the Extracted Folder
old_path = f'/content/{dataset_filename}'
new_path = '/content/dataset'
os.rename(old_path, new_path)

Optional: Check is path is correct

output should be
- train valid
- images labels
- /content/gdrive/MyDrive/vespA/data.yaml


In [ ]:
# Optional chech datapaths

!ls '/content/dataset/'
!ls '/content/dataset/train/'
!ls '/content/dataset/valid/'
!ls '/content/gdrive/MyDrive/vespA/data.yaml'

## 3. Train YOLO11 model

1. Install ultralytics

In [ ]:
#Installing the python package
!pip install ultralytics

#Verifying the installation
!pip show ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import Image

2. Retrain model

In [ ]:
from ultralytics import YOLO

model_name = "yolo11s" #@param {type:"string"}
dataset_path = "/content/gdrive/MyDrive/vespA/data.yaml" #@param {type:"string"}

# Get pre-trained model
model = YOLO(f"{model_name}.pt")

# Train/fine-tune model
model.train(project="yolo11vespA",  # wandb project name
            name="train1",          # run name wandb
            data=dataset_path,
            epochs=3,
            imgsz=640,
            fraction=0.1)

3. Validate model

In [ ]:
!yolo task=detect mode=val \
model="/path/to/YoloRetrained/weights/best.pt" \
    data="/path/to/dataset.yaml"

4. Convert .pt file to ONNX via CLI

In [ ]:
!yolo export model=path/to/best.pt format=onnx  # export custom trained model

OR

convert .pt file to ONNX via python code

In [ ]:
import torch

# Load our model into our environment
checkpoint = torch.load('/path/to/best.pt')
model = checkpoint['model']

model = model.float()
model.eval()

# Dummy input in FP32
dummy_input = torch.randn(16, 3, 640, 640, dtype=torch.float)

# Export to ONNX
torch.onnx.export(
    model,
    dummy_input,
    "modified_run_3.onnx",
    export_params=True,
    opset_version=11,  # Adjust opset version if needed
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output']
)
print("ONNX model exported successfully!")

5. Verify the model

In [ ]:
import onnx
import onnxruntime as ort
import torch

# Load the ONNX model
onnx_model = onnx.load("modified_run_3.onnx")
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")

# Test the ONNX model with ONNX Runtime
dummy_input = torch.randn(16, 3, 640, 640).numpy()
ort_session = ort.InferenceSession("modified_run_3.onnx")
outputs = ort_session.run(None, {"input": dummy_input})
print(outputs[0])

## Hailo dataflow compiler

1. Make virtual environment in Colab

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-dev python3-distutils python3-tk libfuse2 graphviz libgraphviz-dev

# Will need a venv to install the DFC in
!pip install --upgrade pip virtualenv
!virtualenv my_env

Download hailo dataflow compiler (python 3.10) from https://hailo.ai/developer-zone/software-downloads/ (you need to make an account)

Then copy the .whl to google drive


In [ ]:
#Installing the WHL file for Hailo DFC
!my_env/bin/pip install /content/hailo_dataflow_compiler-3.29.0-py3-none-linux_x86_64.whl

# Making sure it's installed properly
!my_env/bin/hailo --version

Identifying the six end node names via www.netron.app

To identify the Yolo’s end nodes, they are the nodes right before the post-processing operations at the very bottom of the model. Their are 2 end nodes per map

In yolov11 model
```
"/model.23/cv2.2/cv2.2.2/Conv",
"/model.23/cv3.2/cv3.2.2/Conv",
"/model.23/cv2.1/cv2.1.2/Conv",
"/model.23/cv3.1/cv3.1.2/Conv",
"/model.23/cv2.0/cv2.0.2/Conv",
"/model.23/cv3.0/cv3.0.2/Conv",
```


2. Parsing to ONNX file

In [ ]:
with open("translate_model.py", "w") as f:
    f.write("""

from hailo_sdk_client import ClientRunner

# Define the ONNX model path and configuration
onnx_path = "/content/modified_run_3.onnx"  # Replace with your ONNX model path
onnx_model_name = "modified_run_3_renamed"
chosen_hw_arch = "hailo8l"  # Specify the target hardware architecture

# Initialize the ClientRunner
runner = ClientRunner(hw_arch=chosen_hw_arch)

# Use the recommended end node names for translation
end_node_names = [
  "/model.23/cv2.2/cv2.2.2/Conv",
  "/model.23/cv3.2/cv3.2.2/Conv",
  "/model.23/cv2.1/cv2.1.2/Conv",
  "/model.23/cv3.1/cv3.1.2/Conv",
  "/model.23/cv2.0/cv2.0.2/Conv",
  "/model.23/cv3.0/cv3.0.2/Conv",
]

try:
    # Translate the ONNX model to Hailo's format
    hn, npz = runner.translate_onnx_model(
        onnx_path,
        onnx_model_name,
        end_node_names=end_node_names,
        net_input_shapes={"images": [1, 3, 320, 320]},  # Adjust input shapes if needed
    )
    print("Model translation successful.")
except Exception as e:
    print(f"Error during model translation: {e}")
    raise

# Save the Hailo model HAR file
hailo_model_har_name = f"{onnx_model_name}_hailo_model.har"
try:
    runner.save_har(hailo_model_har_name)
    print(f"HAR file saved as: {hailo_model_har_name}")
except Exception as e:
    print(f"Error saving HAR file: {e}")

""")

In [ ]:
# Run model in CLI
!my_env/bin/python translate_model.py

3. Optimize model

In [ ]:
from hailo_sdk_client import ClientRunner

# Load the HAR file
har_path = "modified_run_3_renamed_hailo_model.har"

runner = ClientRunner(har=har_path)

from pprint import pprint

try:
    # Access the HailoNet as an OrderedDict
    hn_dict = runner.get_hn()  # Or use runner._hn if get_hn() is unavailable
    print("Inspecting layers from HailoNet (OrderedDict):")

    # Pretty-print each layer
    for key, value in hn_dict.items():
        print(f"Key: {key}")
        pprint(value)
        print("\n" + "="*80 + "\n")  # Add a separator between layers for clarity

except Exception as e:
    print(f"Error while inspecting hn_dict: {e}")

Now, you can scroll through the output to verify which layers correspond to which end node in your ONNX model. In this dict, each layer is stored under a new name, and it’s original name is a key within the layer under ‘original_names’. You will need this when generating a NMS file for your model, you can find examples NMS configs here.

In [ ]:
import json
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive/', force_remount=True)

# Updated NMS layer configuration dictionary
nms_layer_config = {
    "nms_scores_th": 0.3,
    "nms_iou_th": 0.7,
    "image_dims": [640, 640],
    "max_proposals_per_class": 25,
    "classes": 1,
    "regression_length": 16,
    "background_removal": False,
    "background_removal_index": 0,
    "bbox_decoders": [
        {
            "name": "bbox_decoder23",
            "stride": 16,
            "reg_layer": "conv23",
            "cls_layer": "conv26"
        },
        {
            "name": "bbox_decoder38",
            "stride": 32,
            "reg_layer": "conv38",
            "cls_layer": "conv41"
        }
    ]
}

# Path to save the updated JSON configuration
output_dir = "/save/path/"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
output_path = os.path.join(output_dir, "nms_layer_config.json")

# Save the updated configuration as a JSON file
with open(output_path, "w") as json_file:
    json.dump(nms_layer_config, json_file, indent=4)

print(f"NMS layer configuration saved to {output_path}")

After this, I made calibration data for the optimization step.

In [ ]:
import numpy as np
from PIL import Image
import os
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive/', force_remount=True)

# Paths to directories and files
image_dir = '/input/path'
output_dir = '/path/to/output_dir'
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# File paths for saving calibration data
calibration_data_path = os.path.join(output_dir, "calibration_data.npy")
processed_data_path = os.path.join(output_dir, "processed_calibration_data.npy")

# Initialize an empty list for calibration data
calib_data = []

# Process all image files in the directory
for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        img = Image.open(img_path).resize((640, 640))  # Resize to desired dimensions
        img_array = np.array(img) / 255.0  # Normalize to [0, 1]
        calib_data.append(img_array)

# Convert the calibration data to a NumPy array
calib_data = np.array(calib_data)

# Save the normalized calibration data
np.save(calibration_data_path, calib_data)
print(f"Normalized calibration dataset saved with shape: {calib_data.shape} to {calibration_data_path}")

# Scale the normalized data back to [0, 255]
processed_calibration_data = calib_data * 255.0

# Save the processed calibration data
np.save(processed_data_path, processed_calibration_data)
print(f"Processed calibration dataset saved with shape: {processed_calibration_data.shape} to {processed_data_path}")

Now, we’re finally ready to optimize it with this script, you can find sample .alls files here, I referenced yolo10nms.json as a base to create my alls file.

Note that the change_output_activation applied to my CLS_layer, you can go back and verify this with Netron like specified above.



In [ ]:
import os
from hailo_sdk_client import ClientRunner

# Define your model's HAR file name
model_name = "modified_run_3_renamed"
hailo_model_har_name = f"{model_name}_hailo_model.har"
hailo_model_har_name = "modified_run_3_renamed_hailo_model.har"

# Ensure the HAR file exists
assert os.path.isfile(hailo_model_har_name), "Please provide a valid path for the HAR file"

# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=hailo_model_har_name)

# Define the model script to add a normalization layer
# Normalization for [0, 1] range
alls = \"\"\"
normalization1 = normalization([0.0, 0.0, 0.0], [255.0, 255.0, 255.0])
change_output_activation(conv26, sigmoid)
change_output_activation(conv41, sigmoid)
nms_postprocess("/content/nms_layer_config.json", meta_arch=yolov8, engine=cpu)
performance_param(compiler_optimization_level=max)
\"\"\"

# Load the model script into the ClientRunner
runner.load_model_script(alls)

# Define a calibration dataset
# Replace 'calib_dataset' with the actual dataset you're using for calibration
# For example, if it's a directory of images, prepare the dataset accordingly
calib_dataset = "/content/processed_calibration_data.npy"

# Perform optimization with the calibration dataset
runner.optimize(calib_dataset)

# Save the optimized model to a new Quantized HAR file
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

print(f"Quantized HAR file saved to: {quantized_model_har_path}")

Now run it

In [ ]:
!my_env/bin/python optimize_model.py

Compiling model

In [ ]:
from hailo_sdk_client import ClientRunner

# Define the quantized model HAR file
model_name = "modified_run_3_renamed"
quantized_model_har_path = f"{model_name}_quantized_model.har"

# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=quantized_model_har_path)
print("[info] ClientRunner initialized successfully.")

# Compile the model
try:
    hef = runner.compile()
    print("[info] Compilation completed successfully.")
except Exception as e:
    print(f"[error] Failed to compile the model: {e}")
    raise
file_name = f"{model_name}.hef"
with open(file_name, "wb") as f:
    f.write(hef)

now run

In [ ]:
!my_env/bin/python compile_model.py
